In [1]:
from ccDGGeom import np, Point

In [2]:
A = np.array([[4, 3], [6, 5], [2, 1]])
A

array([[4, 3],
       [6, 5],
       [2, 1]])

In [3]:
np.argmin(A)

5

In [4]:
A[:, 0], A[:, 1]

(array([4, 6, 2]), array([3, 5, 1]))

In [5]:
A[:, 0] ** 2 + A[:, 1] ** 2

array([25, 61,  5])

In [6]:
np.argmin(_)

2

In [7]:
points = (Point(4, 3), Point(6, 5), Point(2, 1))
np.stack(tuple(p.tupl for p in points))

array([[4, 3],
       [6, 5],
       [2, 1]])

In [208]:
class Circle:
    
    def __init__(self, x : int, y : int, r : int):
        self.origin = Point(x, y)
        self.radius = abs(r)
        self.edgePoints = {}
        self.refreshEdgePoints()
        
    def refreshEdgePoints(self, charliesMethod : bool = True):
        firstQuarter = []
        p = Point(self.radius, 0)
        while p.x > 0:
            firstQuarter.append(p)
            nextPoints = (Point(p.x - 1, p.y), Point(p.x, p.y + 1), Point(p.x - 1, p.y + 1))
            XY = np.stack(tuple(p.tupl for p in nextPoints))
            
            if charliesMethod:
                nextPointsWithinRange = []
                for i in np.where(
                    XY[:, 0] ** 2 + XY[:, 1] ** 2 - (
                        self.radius ** 2 + int(self.radius ** 0.5)
                    ) <= 0
                )[0]:
                    nextPointsWithinRange.append(nextPoints[i])
                    
                XYWithinRange = np.stack(tuple(p.tupl for p in nextPointsWithinRange))
                
                p = nextPointsWithinRange[
                    np.argmax(
                        XYWithinRange[:, 0] ** 2 + XYWithinRange[:, 1] ** 2
                    )
                ]
            else:
                
                p = nextPoints[
                    np.argmin(
                        np.abs(
                            XY[:, 0] ** 2 + XY[:, 1] ** 2 - (
                                self.radius ** 2 + int(self.radius ** 0.5)
                            )
                        )
                    )
                ]
        
        fqA = np.array(tuple(p.tupl for p in firstQuarter), int).transpose()
        sqR, tqR, rqR = (
            np.array([[np.cos(t), -np.sin(t)], [np.sin(t), np.cos(t)]])
            for t in (np.pi / 2, np.pi, 3 * np.pi / 2)
        )
        secondQuarter = (sqR @ fqA).round().astype(int).transpose().tolist()
        thirdQuarter = (tqR @ fqA).round().astype(int).transpose().tolist()
        fourthQuarter = (rqR @ fqA).round().astype(int).transpose().tolist()
        
        self.edgePoints = {p + self.origin for p in firstQuarter} \
            | {Point(*p) + self.origin for p in secondQuarter} \
            | {Point(*p) + self.origin for p in thirdQuarter} \
            | {Point(*p) + self.origin for p in fourthQuarter}
        
    def getMinFrame(self) -> Point:
        return self.origin + Point(self.radius + 1, self.radius + 1)
        
    def getMaskEdge(self, fw : int = 0, fh : int = 0) -> np.array:
        if fw == 0 or fh == 0:
            f = self.getMinFrame()
            fw = f.x
            fh = f.y
            
        M = np.zeros((fh, fw), bool)
        
        for p in self.edgePoints:
            M[p.y, p.x] = 1
            
        return M
    
    def getMaskFill(self, fw : int = 0, fh : int = 0) -> np.array:
        M = self.getMaskEdge(fw, fh)
        
        queue = [self.origin]
        while len(queue) > 0:
            p = queue.pop()
            if M[p.y, p.x] == 0:
                M[p.y, p.x] = 1
                queue.insert(0, Point(p.x, p.y - 1))
                queue.insert(0, Point(p.x - 1, p.y))
                queue.insert(0, Point(p.x, p.y + 1))
                queue.insert(0, Point(p.x + 1, p.y))
                
        return M
    

In [209]:
np.maximum((-1, 2, -3), 0)

array([0, 2, 0])

In [210]:
np.where(np.array((-1, 2, -3, -4, 5), int) > 0)[0]

array([1, 4])

In [232]:
r = 8

In [233]:
c = Circle(15, 15, r)
#c.refreshEdgePoints(charliesMethod=False)
c.getMaskEdge()

array([[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, #, #, #, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, #, #, #, _, _, _, #, #, #, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, #, _, _, _, _, _, _, _, _, _, #, _, _, _],
       [_, _, _, _, _, _, _, _, _, #, _, _, _, _, _, _, _, _, _, _, _, #, _, _],
       [_, _, _, _, _, _, _, _, #, _, _, _, _, _, _, _, _, _, _, _, _, _, #, _],
       [_, _, _, _, _, _, _,

In [234]:
c.getMaskFill()

array([[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, #, #, #, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, #, #, #, #, #, #, #, #, #, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, #, #, #, #, #, #, #, #, #, #, #, _, _, _],
       [_, _, _, _, _, _, _, _, _, #, #, #, #, #, #, #, #, #, #, #, #, #, _, _],
       [_, _, _, _, _, _, _, _, #, #, #, #, #, #, #, #, #, #, #, #, #, #, #, _],
       [_, _, _, _, _, _, _,

In [235]:
tuple(c for c in "foobar")

('f', 'o', 'o', 'b', 'a', 'r')

In [236]:
type(_)

tuple

In [237]:
c2 = Circle(15, 15, r)
c2.refreshEdgePoints(charliesMethod=False)
c2.getMaskEdge() | c.getMaskEdge()

array([[_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, _, #, #, #, #, #, #, #, _, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, _, #, #, #, _, _, _, #, #, #, _, _, _, _],
       [_, _, _, _, _, _, _, _, _, _, #, _, _, _, _, _, _, _, _, _, #, _, _, _],
       [_, _, _, _, _, _, _, _, _, #, _, _, _, _, _, _, _, _, _, _, _, #, _, _],
       [_, _, _, _, _, _, _, _, #, _, _, _, _, _, _, _, _, _, _, _, _, _, #, _],
       [_, _, _, _, _, _, _,